Welcome to the demo of queer supercrystals in SageMath.

Firstly, one can create queer crystal on letters of type $q(2+1)$

In [0]:
Q = crystals.Letters(['Q',3]); Q

Then, one can create a crystal on words (length 6) and obtain its index set.

In [0]:
T = tensor([Q]*6)
T.index_set()

The following lists all highest weight elements in this crystal.

In [0]:
[t for t in T if all(t.epsilon(i)==0 for i in Q.index_set())]

One can view the associated crystal graph in a pop up window. This feature requires tikz and dot2tex.

In [0]:
# local hack to show latex figures
# on local version of Sage, use view
from viewLatex import viewLatex

In [0]:
Q = crystals.Letters(['Q',3])
T = tensor([Q]*2)
viewLatex(T)

One can obtain the LaTeX commands for the crystal graphs to be included in .tex files

In [0]:
latex(T)

One can restrict to a subcrystal given a crystal element and subset of indices. 

In [0]:
C = T.subcrystal(generators=[T(Q(1),Q(2),Q(1),Q(1))], index_set=[1,2,-1])
viewLatex(C)

Full options for subcrystal are available in documentation. Run ? after a method, attribute or object to see its documentation.


In [0]:
T.subcrystal?

One can write extra code to construct a type $A_n$ component graph given a dominant weight.

In [0]:
def reading_word_highest(P,n):
    """
    Returns a highest weight word given a strict partition P and n.
    Note that n must be at least the number of parts of P.

    EXAMPLE::

    sage: reading_word_highest([5,3,2],4)
    [1,1,2,3,3,1,2,2,1,1]

    """
    if n < len(P):
        raise ValueError("n must be at least %s" %len(P))
    Q = crystals.Letters(['Q',n])
    T = tensor([Q]*sum(P))
    L = P+[0]
    w = []
    for m in range(len(P)):
        for k in range(len(P)-m):
            w += [Q(k+1)]*(L[m+k]-L[m+k+1])
    return T(*w)

def G_C_graph(P,n):
    """
    Creates a graph `G(C)` of type `A_n` component graphs given strict partition P.
    Note that n must be at least len(P).

    EXAMPLES::

        sage: P = Partition([5,2])
        sage: G_C_graph(P,5)
        Digraph on 13 vertices
    """
    if len([i for i in range(len(P)-1) if P[i]<=P[i+1]]) > 0:
        raise ValueError("P must be a strict partition")
    if n < len(P):
        raise ValueError("n must be at least len(P)")
    I = range(1,n)
    t = reading_word_highest(P,n)
    V = [t]
    j = 0
    while j<len(V):
        u = V[j]
        L = [(i,h) for i in I for h in I+[n] if h>i and lowering(u,i,h)!=None]
        for l in L:
            i,h = l
            v = lowering(u,i,h).f(-i)
            if v != None and v.is_highest_weight(index_set=I)==True:
                if v not in V:
                    V += [v]
        j += 1

    E = set([])
    j = 0
    while j<len(V):
        v = V[j]
        I = range(1,n)
        for i in I:
            u = v.e(-i)
            if u != None:
                u = u.to_highest_weight(index_set=I)[0]
                E.add(tuple([u,v]))
        j += 1

    G = DiGraph([sorted(V),sorted(list(E))],format='vertices_and_edges')
    G.set_latex_options(format='dot2tex')
    return G

def lowering(t,i,h):
    """
    Computes the composition f_{h-1}...f_{i+1} of t, where i<h.

    EXAMPLES::
        sage: Q = crystals.Letters(['Q',5])
        sage: T = tensor([Q]*6)
        sage: u = T(Q(4),Q(3),Q(2),Q(1),Q(2),Q(1))
        sage: lowering(u,2,5)
        [5, 4, 2, 1, 2, 1]
    """
    if h<=i:
        raise ValueError("h must be greater than i")
    w = t
    for j in range(-h+1,-i):
        w = w.f(-j)
        if w == None:
            return None
    return w

The following blocks runs the example of a type $A_n$ graph for dominant weight (5,2,0,0,0).

In [0]:
P = Partition([5,2])
G = G_C_graph(P,5)
viewLatex(G)